<a href="https://colab.research.google.com/github/Xinxinatg/CSRNet-pytorch/blob/master/make_dataset_and_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!git clone https://github.com/vivek-bombatkar/CSRNet-pytorch.git

In [ ]:
#!wget --header="Host: doc-0k-64-docs.googleusercontent.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-AU,en;q=0.9,zh-CN;q=0.8,zh;q=0.7,en-GB;q=0.6,en-US;q=0.5" --header="Cookie: AUTH_9hg3ee1ib7ptld264s2flqgrf9hg9jro_nonce=lrmfhnf36dg1o" --header="Connection: keep-alive" "https://doc-0k-64-docs.googleusercontent.com/docs/securesc/0chsofepnd39clr7fg0d2tu89snbaa74/9bol3t2b507br9r703uqsf0vv5ejk3rg/1603972950000/07527692003091305349/07527692003091305349/1tGszHbRCFg4NwlrMo9p1GrPl7TaBzxsK?e=download&authuser=0&nonce=lrmfhnf36dg1o&user=07527692003091305349&hash=5qpp4gg2o4orl8c2f2sacamrj63t6a64" -c -O 'VisDrone2020-CC_processed.zip'

In [ ]:
#!unzip -q /content/VisDrone2020-CC_processed.zip

In [ ]:
import h5py
import json
import scipy.io as io
import sys
sys.path.extend(['/content/CSRNet-pytorch'])
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter 
import scipy
import scipy.spatial
import json
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
%matplotlib inline

In [ ]:
#this is borrowed from https://github.com/davideverona/deep-crowd-counting_crowdnet
def gaussian_filter_density(gt):
    print (gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density

#    pts = np.array(zip(np.nonzero(gt)[1], np.nonzero(gt)[0]))  python 2 version
    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    tree = scipy.spatial.KDTree(pts.copy(), leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)

    print ('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1],pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1]+distances[i][2]+distances[i][3])*0.1
        else:
            sigma = np.average(np.array(gt.shape))/2./2. #case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print ('done.')
    return density

In [ ]:
#set the root to the Shanghai dataset you download
root = '/content/VisDrone2020-CC/'

In [ ]:
#now generate the ShanghaiA's ground truth
#                part_A_train = os.path.join(root,'part_A_final/train_data','images')
 #               part_A_test = os.path.join(root,'part_A_final/test_data','images')
  #              part_B_train = os.path.join(root,'part_B_final/train_data','images')
   #             part_B_test = os.path.join(root,'part_B_final/test_data','images')
    #            path_sets = [part_A_train,part_A_test]
train = os.path.join(root,'train','images')
test = os.path.join(root,'test','images')
path_sets = [train,test]

In [ ]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
        img_paths.append(img_path)

with open('train.json', 'w') as f:
    json.dump(img_paths, f)

In [ ]:
for img_path in img_paths:
    print (img_path)
    mat = io.loadmat(img_path.replace('.jpg','.mat').replace('images','ground_truth').replace('IMG_','GT_IMG_'))
    img= plt.imread(img_path)
    k = np.zeros((img.shape[0],img.shape[1]))
    gt = mat["image_info"][0,0][0,0][0]
    for i in range(0,len(gt)):
        if int(gt[i][1])<img.shape[0] and int(gt[i][0])<img.shape[1]:
            k[int(gt[i][1]),int(gt[i][0])]=1
    k = gaussian_filter_density(k)
    with h5py.File(img_path.replace('.jpg','.h5').replace('images','ground_truth'), 'w') as hf:
            hf['density'] = k

/content/VisDrone2020-CC/train/images/IMG_00064_00004.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00084_00012.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00051_00014.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00002_00025.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00066_00019.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00005_00029.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00013_00015.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00005_00016.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00087_00030.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train/images/IMG_00064_00018.jpg
(1080, 1920)
generate density...
done.
/content/VisDrone2020-CC/train

In [ ]:
#now see a sample from ShanghaiA
plt.imshow(Image.open(img_paths[0]))

In [ ]:
gt_file = h5py.File(img_paths[0].replace('.jpg','.h5').replace('images','ground_truth'),'r')
groundtruth = np.asarray(gt_file['density'])
plt.imshow(groundtruth,cmap=CM.jet)

In [ ]:
np.sum(groundtruth)# don't mind this slight variation

In [ ]:
!cd CSRNet_pytorch
!python train.py /content/CSRNet-pytorch/train.json /content/CSRNet-pytorch/val.json 0 0